In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import metrics
import myfm
from myfm import RelationBlock
import pandas as pd
from scipy import sparse as sps
from mapper import DefaultMapper

# read movielens 100k data.
from movielens100k_data import MovieLens100kDataManager

In [2]:
data_manager = MovieLens100kDataManager()
df_train, df_test = data_manager.load_rating(fold=1)

user_info = data_manager.load_userinfo().set_index('user_id')
user_info['age'] = user_info.age // 10 * 10
user_info['zipcode'] = user_info.zipcode.str[0]

user_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(user_info)

In [3]:
user_info.head()

,age,gender,occupation,zipcode
user_id,,,,
1,20,M,technician,8
2,50,F,other,9
3,20,M,writer,3
4,20,M,technician,4
5,30,F,other,1


In [4]:
user_to_internal = DefaultMapper(df_train.user_id.values)
movie_to_internal = DefaultMapper(df_train.movie_id)

In [5]:
date_be = OneHotEncoder(handle_unknown='ignore').fit(
    df_train.timestamp.dt.date.values.reshape(-1, 1)
)
def categorize_date(df):
    return date_be.transform(df.timestamp.dt.date.values[:, np.newaxis])

In [6]:
X_date_train = categorize_date(df_train)
X_date_test  = categorize_date(df_test)

In [7]:
# Implement flavor of SVD++
# We add "all users who have evaluated a movie in the train set" or
# "all movies rated by a user" as a feture of user/movie.
use_iu = True # use implicit user feature
use_ii = True # use implicit item feature
use_user_info = False # use user information

movie_vs_watched = dict()
user_vs_watched = dict()
for row in df_train.itertuples():
    user_id = row.user_id
    movie_id = row.movie_id
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)

In [8]:
# setup grouping
feature_group_sizes = [
    len(date_be.categories_[0]), # date
    len(user_to_internal) # user ids
] 
if use_iu:
    feature_group_sizes.append(len(movie_to_internal))

if use_user_info:
    feature_group_sizes.extend([
        len(c) for c in user_info_ohe.categories_ # user attributes
    ])

feature_group_sizes.append(
    len(movie_to_internal) # movie ids
)
if use_ii:
    feature_group_sizes.append(
        len(user_to_internal) # all users who watched the movies
    )

grouping = [ i for i, size in enumerate(feature_group_sizes) for _ in range(size)]

In [9]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    if use_iu:
        X = sps.lil_matrix((len(user_ids), len(user_to_internal) + len(movie_to_internal)))
    else:
        X = sps.lil_matrix((len(user_ids), len(user_to_internal)))
    for index, user_id in enumerate(user_ids):
        X[index, user_to_internal[user_id]] = 1
        if not use_iu:
            continue
        watched_movies = user_vs_watched.get(user_id, [])
        normalizer = 1 / max(len(watched_movies), 1) ** 0.5
        for uid in watched_movies:
            X[index, movie_to_internal[uid] + len(user_to_internal)] = normalizer
    if use_user_info:
        return sps.hstack(
            [X.tocsr(), user_info_ohe.transform(user_info.reindex(user_ids))],
            format='csr'
        )
    else:
        return X.tocsr()

def augment_movie_id(movie_ids):
    if use_ii:
        X = sps.lil_matrix((len(movie_ids), len(movie_to_internal)+len(user_to_internal)))
    else:
        X = sps.lil_matrix((len(movie_ids), len(movie_to_internal)))
    for index, movie_id in enumerate(movie_ids):
        X[index, movie_to_internal[movie_id]] = 1
        if not use_ii:
            continue
        watched_users = movie_vs_watched.get(movie_id, [])
        normalizer = 1 / max(len(watched_users), 1) ** 0.5
        for uid in watched_users:
            X[index, user_to_internal[uid] + len(movie_to_internal)] = normalizer
    return X.tocsr()

In [10]:
def get_key_and_index(keys):
    unique_keys = np.unique(keys)
    to_index = { key: i for i, key in enumerate(unique_keys)}
    index = np.asarray([to_index[j] for j in keys])
    return unique_keys, index

## User Relation Block to express data
See [\[Rendle 2013\]](http://www.vldb.org/pvldb/vol6/p337-rendle.pdf) how comlexity dcrease drastically in this case (and most cases with bipartite graph structure).

In [11]:
# Create RelationBlock.
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks), (df_test, test_blocks)]:
    unique_users, user_map = get_key_and_index(source.user_id) 
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = get_key_and_index(source.movie_id)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

In [12]:
fm = myfm.MyFMRegressor(rank=8)
fm.fit(X_date_train, df_train.rating.values, X_rel=train_blocks,
        grouping=grouping, X_test=X_date_test, X_rel_test=test_blocks,
        y_test=df_test.rating.values,
        n_kept_samples=195, n_iter=200);

alpha = 1.58 w0 = 3.04  rmse_this: 0.96 mae_this: 0.75: 100%|██████████| 200/200 [00:08<00:00, 22.29it/s]


In [13]:
test_predictions = fm.predict(X_date_test, test_blocks)

In [14]:
rmse = (
    (test_predictions - df_test.rating.values)**2
).mean() ** 0.5
mae = np.abs(test_predictions - df_test.rating).mean()

# Note the improvement from "id_only" case.
# Compare this with methods like ones in https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k
print('rmse={}, mae={}'.format(rmse, mae))

rmse=0.8883743119084084, mae=0.6985641283793332


In [15]:
# If we use the original data format, it takes much more!
X_original_format = sps.hstack([
    X_date_train
] + [rel.data[rel.original_to_block] for rel in train_blocks], format='csr')

fm_rawformat = myfm.MyFMRegressor(rank=8).fit(X_original_format, df_train.rating,
                                          grouping=grouping, n_iter=6, n_kept_samples=1)

alpha = 1.25 w0 = 3.41 : 100%|██████████| 6/6 [00:10<00:00,  1.83s/it]


In [16]:
# They shoud be same up to floating point artifact.
fm_rawformat.predictor_.samples[-1].V - fm.predictor_.samples[0].V

array([[-2.66453526e-15,  3.33066907e-16, -2.49800181e-15, ...,
        -3.60822483e-16,  1.60982339e-15,  4.10782519e-15],
       [ 5.77315973e-15,  5.49560397e-15,  1.69309011e-15, ...,
         3.05311332e-15,  2.27595720e-15, -2.35228503e-15],
       [-1.05020159e-14, -5.10702591e-15, -4.85722573e-16, ...,
         2.10942375e-15,  1.36002321e-15,  1.54043445e-15],
       ...,
       [ 8.60422844e-16,  1.05471187e-15,  4.16333634e-16, ...,
         2.77555756e-17, -5.55111512e-17, -4.99600361e-16],
       [ 8.60422844e-16,  1.76247905e-15,  1.17614252e-15, ...,
        -1.16573418e-15,  8.74300632e-16,  6.38378239e-16],
       [ 1.36002321e-15, -2.77555756e-15,  1.77635684e-15, ...,
         2.49800181e-16,  3.33066907e-16,  3.23352456e-15]])